# ANN- Classification analysis on **Churn modelling**

## Importing Libraries

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf


## Data preprocessing

### Importing data set

In [15]:
df=pd.read_csv('/Churn_Modelling.csv')

In [16]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [22]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [18]:
x=df.iloc[:,3:-1].values
y=df.iloc[:,-1].values

In [19]:
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

### Encoding categorical data

Label Encoding the "Gender" column

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

One Hot Encoding the "Geography" column

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [28]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [29]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

### Feature Scaling

In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Building ANN

### Initializing ANN

In [33]:
model=tf.keras.Sequential()

### Adding the input layer and the first hidden layer

In [34]:
model.add(tf.keras.layers.Dense(6,activation='relu'))

### Adding the second hidden layer

In [35]:
model.add(tf.keras.layers.Dense(6,activation='relu'))

### Adding the output layer

In [36]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

Note: we did not add any activation function to the output layer because we need continuous value after the weighted sum

## Part 3 - Training the ANN

### Compiling the ANN

In [37]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

### Training the ANN model on the Training set

In [38]:
model.fit(x_train,y_train,epochs=100,batch_size=30)

Epoch 1/100
267/267 [==============================] - 1s 1ms/step - loss: 0.6876 - accuracy: 0.5356
Epoch 2/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4962 - accuracy: 0.7957
Epoch 3/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4524 - accuracy: 0.7986
Epoch 4/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4403 - accuracy: 0.8032
Epoch 5/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.8160
Epoch 6/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4236 - accuracy: 0.8094
Epoch 7/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4291 - accuracy: 0.8107
Epoch 8/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4098 - accuracy: 0.8179
Epoch 9/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4117 - accuracy: 0.8233
Epoch 10/100
267/267 [==============================] - 0s 1ms/step - loss: 0.4016 - accura

## Predicting the result of a single observation



Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [39]:
print(model.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

### Predicting the Test set results

In [41]:
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusion Matrix

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
f1_score(y_test,y_pred)

[[1513   82]
 [ 195  210]]
0.8615


0.6025824964131994

F1 score is 0.6 which means our model is not performing well, but it is acceptable.